In [91]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import json
import re
import json
import glob
import datetime
from collections import Counter

from datetime import datetime
import math
import scipy
import scipy.stats
import panel as pn
pn.extension()
from vega_datasets import data as vds

In [92]:
# Gerando os URL dos .csv
caminho = r'dados/dados-sp-'
todos_dados_csv = glob.glob(caminho + "*.csv")

# Colunas que incluem datas
datas = ['dataNotificacao', 'dataInicioSintomas', 'dataTeste', 'dataEncerramento']

# Colunas a serem lidas
cols = ["dataNotificacao", "dataInicioSintomas", "sintomas", "profissionalSaude", "cbo","condicoes",
        "dataTeste", "tipoTeste", "resultadoTeste", "sexo", "municipio", "municipioIBGE", "estadoNotificacao",
        "estadoNotificacaoIBGE", "idade", "dataEncerramento", "evolucaoCaso", "classificacaoFinal"]

# Carregando os dados
dados = pd.concat((pd.read_csv(f, sep=';', encoding='latin1', parse_dates=datas, usecols=cols, infer_datetime_format = False) for f in todos_dados_csv), ignore_index = True)

In [93]:
# Exclui as linhas cujas colunas indicadas abaixo sao null
dados.dropna(subset = ["tipoTeste", "resultadoTeste", "classificacaoFinal"], inplace = True)

# Exclui os dados Cancelados
dados.drop(dados[dados['evolucaoCaso'] == 'Cancelado'].index, inplace = True)

# Exclui os dados que possuem datas fora do escopo esperado
data_limite_inferior = np.datetime64('2020-01-01', 'ns') # Comeco da pandemia
data_limite_superior = np.datetime64('2021-03-25', 'ns') # A publicacao dos dados mais recentes ocorreram nesse dia
for data in datas:
    dados[data] = pd.to_datetime(dados[data], utc=True).dt.tz_localize(None) # Retira o fuso horario
    dados.drop(dados[(dados[data] < data_limite_inferior) | (dados[data] > data_limite_superior)].index, inplace = True) # Exclui

dados[dados['idade'] > 116] = float("NaN") # Exclui idades invalidas
dados.reset_index(drop=True, inplace = True) # Realoca os indices depois das exclusoes

In [94]:
cidades = {}
for i in range(len(dados)):
    cidadeIBGE = dados['municipioIBGE'][i]
    if cidadeIBGE != cidadeIBGE: # Precisa disso pra pular os dados cujo campo municipioIBGE é NULL (float vira NaN)
        continue
    cidadeIBGE = int(cidadeIBGE)
    resultadoTeste = dados['resultadoTeste'][i]
    if cidadeIBGE not in cidades:
        cidades[cidadeIBGE] = {}
        cidades[cidadeIBGE]['Nome'] = dados['municipio'][i]
        cidades[cidadeIBGE]['Positivo'] = 0
        cidades[cidadeIBGE]['Negativo'] = 0
        cidades[cidadeIBGE]['Inconclusivo ou Indeterminado'] = 0

    # Precisa disso por causa dos NaN
    if resultadoTeste not in cidades[cidadeIBGE]:
        cidades[cidadeIBGE][resultadoTeste] = 0
    cidades[cidadeIBGE][resultadoTeste] += 1

In [96]:
#Número total de testes realizados e que apresentam uma classificação final
total_testes = dados.shape[0]


#Número de falsos negativos dos testes
teste_RT_falsoneg = len(dados.query('tipoTeste == "RT-PCR" & resultadoTeste == "Negativo" & (classificacaoFinal == "Confirmado Laboratorial" | classificacaoFinal =="Confirmado Clínico-Epidemiológico" |classificacaoFinal == "Confirmado Clínico-Imagem"| classificacaoFinal =="Confirmado por Critério Clínico" | classificacaoFinal == "Confirmação Laboratorial")'))

teste_anticorpo_falsoneg = len(dados.query('tipoTeste == "TESTE RÁPIDO - ANTICORPO" & resultadoTeste == "Negativo" & (classificacaoFinal == "Confirmado Laboratorial" | classificacaoFinal =="Confirmado Clínico-Epidemiológico" |classificacaoFinal == "Confirmado Clínico-Imagem"| classificacaoFinal =="Confirmado por Critério Clínico" | classificacaoFinal == "Confirmação Laboratorial")'))

teste_antigeno_falsoneg = len(dados.query('tipoTeste == "TESTE RÁPIDO - ANTÍGENO" & resultadoTeste == "Negativo" & (classificacaoFinal == "Confirmado Laboratorial" | classificacaoFinal =="Confirmado Clínico-Epidemiológico" |classificacaoFinal == "Confirmado Clínico-Imagem"| classificacaoFinal =="Confirmado por Critério Clínico" | classificacaoFinal == "Confirmação Laboratorial")'))

teste_ELISA_falsoneg = len(dados.query('(tipoTeste == "Enzimaimunoensaio - ELISA IgM" | tipoTeste == "Enzimaimunoensaio  ELISA") & resultadoTeste == "Negativo" & (classificacaoFinal == "Confirmado Laboratorial" | classificacaoFinal =="Confirmado Clínico-Epidemiológico" |classificacaoFinal == "Confirmado Clínico-Imagem"| classificacaoFinal =="Confirmado por Critério Clínico" | classificacaoFinal == "Confirmação Laboratorial")'))

teste_CLIA_falsoneg = len(dados.query('(tipoTeste == "Quimioluminescência - CLIA" | tipoTeste == "Imunoensaio por Eletroquimioluminescência  ECLIA") & resultadoTeste == "Negativo" & (classificacaoFinal == "Confirmado Laboratorial" | classificacaoFinal =="Confirmado Clínico-Epidemiológico" |classificacaoFinal == "Confirmado Clínico-Imagem"| classificacaoFinal =="Confirmado por Critério Clínico" | classificacaoFinal == "Confirmação Laboratorial")'))

teste_ECLIA_falsoneg = len(dados.query('tipoTeste == "Imunoensaio por Eletroquimioluminescência - ECLIA IgG" & resultadoTeste == "Negativo" & (classificacaoFinal == "Confirmado Laboratorial" | classificacaoFinal =="Confirmado Clínico-Epidemiológico" |classificacaoFinal == "Confirmado Clínico-Imagem"| classificacaoFinal =="Confirmado por Critério Clínico" | classificacaoFinal == "Confirmação Laboratorial")'))


#Número de falsos positivos dos testes
teste_RT_falsoposi = len(dados.query('tipoTeste == "RT-PCR" & resultadoTeste == "Positivo" & (classificacaoFinal == "Descartado" | classificacaoFinal == "Síndrome Gripal Não Especificada")'))

teste_anticorpo_falsoposi = len(dados.query('tipoTeste == "TESTE RÁPIDO - ANTICORPO" & resultadoTeste == "Positivo" &(classificacaoFinal == "Descartado" | classificacaoFinal == "Síndrome Gripal não Especificada")'))

teste_antigeno_falsoposi = len(dados.query('tipoTeste == "TESTE RÁPIDO - ANTÍGENO" & resultadoTeste == "Positivo" & (classificacaoFinal == "Descartado" | classificacaoFinal == "Síndrome Gripal não Especificada")'))

teste_ELISA_falsoposi = len(dados.query('(tipoTeste == "Enzimaimunoensaio - ELISA IgM" | tipoTeste == "Enzimaimunoensaio  ELISA") & resultadoTeste == "Positivo" & (classificacaoFinal == "Descartado" | classificacaoFinal == "Síndrome Gripal não Especificada")'))

teste_CLIA_falsoposi = len(dados.query('tipoTeste == "Quimioluminescência - CLIA"  & resultadoTeste == "Positivo" & (classificacaoFinal == "Descartado" | classificacaoFinal == "Síndrome Gripal não Especificada")'))

teste_ECLIA_falsoposi = len(dados.query('tipoTeste == "Imunoensaio por Eletroquimioluminescência - ECLIA IgG" & resultadoTeste == "Positivo" & (classificacaoFinal == "Descartado" | classificacaoFinal == "Síndrome Gripal não Especificada")'))


#Número total de testes realizados e que apresentam uma classificação final
total_RT = len(dados.query('tipoTeste == "RT-PCR"'))

total_anticorpo = len(dados.query('tipoTeste == "TESTE RÁPIDO - ANTICORPO"'))

total_antigeno = len(dados.query('tipoTeste == "TESTE RÁPIDO - ANTÍGENO" '))

total_ELISA = len(dados.query('(tipoTeste == "Enzimaimunoensaio - ELISA IgM" | tipoTeste == "Enzimaimunoensaio  ELISA")'))

total_CLIA = len(dados.query('tipoTeste == "Quimioluminescência - CLIA" '))

total_ECLIA = len(dados.query('tipoTeste == "Imunoensaio por Eletroquimioluminescência - ECLIA IgG" | tipoTeste == "Imunoensaio por Eletroquimioluminescência  ECLIA"'))

probabilidade_falsoneg = [teste_RT_falsoneg/total_RT, teste_anticorpo_falsoneg/total_anticorpo, teste_antigeno_falsoneg/total_antigeno, teste_ELISA_falsoneg/total_ELISA, teste_CLIA_falsoneg/total_CLIA, teste_ECLIA_falsoneg/total_ECLIA]

probabilidade_falsoposi = [teste_RT_falsoposi/total_RT, teste_anticorpo_falsoposi/total_anticorpo, teste_antigeno_falsoposi/total_antigeno, teste_ELISA_falsoposi/total_ELISA, teste_CLIA_falsoposi/total_CLIA, teste_ECLIA_falsoposi/total_ECLIA]

print("Porcentagem de falsos negativos para cada teste:\n")
print('Teste RT-PCR : ',probabilidade_falsoneg[0] *100,'%')
print('Teste Rápido - Anticorpo : ',probabilidade_falsoneg[1]*100,'%')
print('Teste Rápido - Antígeno :',probabilidade_falsoneg[2]*100,'%')
print('Teste Enzimaimunoensaio - ELISA : ',probabilidade_falsoneg[3]*100,'%')
print('Teste Quimioluminescência - CLIA : ',probabilidade_falsoneg[4]*100,'%')
print('Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : ',probabilidade_falsoneg[5]*100,'%')

print("\nPorcentagem de falsos positivos para cada teste:\n")

print('Teste RT-PCR : ',probabilidade_falsoposi[0] *100,'%')
print('Teste Rápido - Anticorpo : ',probabilidade_falsoposi[1]*100,'%')
print('Teste Rápido - Antígeno :',probabilidade_falsoposi[2]*100,'%')
print('Teste Enzimaimunoensaio - ELISA : ',probabilidade_falsoposi[3]*100,'%')
print('Teste Quimioluminescência - CLIA : ',probabilidade_falsoposi[4]*100,'%')
print('Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : ',probabilidade_falsoposi[5]*100,'%')

Porcentagem de falsos negativos para cada teste:

Teste RT-PCR :  5.564918762425288 %
Teste Rápido - Anticorpo :  5.69181679725277 %
Teste Rápido - Antígeno : 4.715533088235294 %
Teste Enzimaimunoensaio - ELISA :  9.175 %
Teste Quimioluminescência - CLIA :  4.661432777232581 %
Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG :  1.546475663356666 %

Porcentagem de falsos positivos para cada teste:

Teste RT-PCR :  0.4610868291264833 %
Teste Rápido - Anticorpo :  0.37548571660209745 %
Teste Rápido - Antígeno : 0.4535845588235294 %
Teste Enzimaimunoensaio - ELISA :  0.25 %
Teste Quimioluminescência - CLIA :  0.4416094210009814 %
Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG :  0.37440989744424547 %


In [204]:
#Número de testes inconclusivos ou indeterminados
print(dados['resultadoTeste'].value_counts()[2]/(dados['resultadoTeste'].value_counts()[0]+dados['resultadoTeste'].value_counts()[1]+dados['resultadoTeste'].value_counts()[2]) *100)

0.15597501216641824


In [206]:
def erro_otimista(alpha, probabilidade, total_casos):
    return abs(scipy.stats.norm.ppf(alpha/2)) *  math.sqrt(probabilidade *(1 - probabilidade)/total_casos)

def erro_conservativo(alpha, total_casos):
    return abs(scipy.stats.norm.ppf(alpha/2)) * (1/math.sqrt(4 * total_casos))

# A partir da análise pontual podemos encontrar a estimativa intervalar para cada teste com alpha = 1% = 0.01
alpha = 0.01

#Fazendo a análise para os casos falsos negativos

#Para o caso da abordagem otimista
erros_otimistas_falsoneg = [erro_otimista(alpha, probabilidade_falsoneg[0], total_RT),erro_otimista(alpha, probabilidade_falsoneg[1], total_anticorpo),
                   erro_otimista(alpha, probabilidade_falsoneg[2], total_antigeno), erro_otimista(alpha, probabilidade_falsoneg[3], total_ELISA),
                   erro_otimista(alpha, probabilidade_falsoneg[4], total_CLIA), erro_otimista(alpha, probabilidade_falsoneg[5], total_ECLIA)]

#Estimativa intervalar para a abordagem otimista
print('Considerando a abordagem otimista\n')

print('Teste RT-PCR : [',(probabilidade_falsoneg[0] - erros_otimistas_falsoneg[0])*100,";",(probabilidade_falsoneg[0] + erros_otimistas_falsoneg[0])*100,']')

print('Teste Rápido - Anticorpo : [',(probabilidade_falsoneg[1] - erros_otimistas_falsoneg[1])*100,";",(probabilidade_falsoneg[1] + erros_otimistas_falsoneg[1])*100,']')

print('Teste Rápido - Antígeno : [',(probabilidade_falsoneg[2] - erros_otimistas_falsoneg[2])*100,";",(probabilidade_falsoneg[2] + erros_otimistas_falsoneg[2])*100,']')

print('Teste Enzimaimunoensaio - ELISA : [',(probabilidade_falsoneg[3] - erros_otimistas_falsoneg[3])*100,";",(probabilidade_falsoneg[3] + erros_otimistas_falsoneg[3])*100,']')

print('Teste Quimioluminescência - CLIA : [',(probabilidade_falsoneg[4] - erros_otimistas_falsoneg[4])*100,";",(probabilidade_falsoneg[4] + erros_otimistas_falsoneg[4])*100,']')

print('Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : [',(probabilidade_falsoneg[5] - erros_otimistas_falsoneg[5])*100,";",(probabilidade_falsoneg[5] + erros_otimistas_falsoneg[5])*100,']')

#Para o caso da abordagem conservativa
erros_conservativos_falsoneg = [erro_conservativo(alpha, total_RT), erro_conservativo(alpha, total_anticorpo), 
                                erro_conservativo(alpha, total_antigeno), erro_conservativo(alpha, total_ELISA),
                                erro_conservativo(alpha, total_CLIA), erro_conservativo(alpha, total_ECLIA)]


#Estimativa intervalar para a abordagem conservativa
print('\nConsiderando a abordagem conservativa\n')

print('Teste RT-PCR : [',(probabilidade_falsoneg[0] - erros_conservativos_falsoneg[0])*100,";",(probabilidade_falsoneg[0] + erros_conservativos_falsoneg[0])*100,']')

print('Teste Rápido - Anticorpo : [',(probabilidade_falsoneg[1]  - erros_conservativos_falsoneg[1])*100,";",(probabilidade_falsoneg[1]  + erros_conservativos_falsoneg[1])*100,']')

print('Teste Rápido - Antígeno : [',(probabilidade_falsoneg[2]  - erros_conservativos_falsoneg[2])*100,";",(probabilidade_falsoneg[2] + erros_conservativos_falsoneg[2])*100,']')

print('Teste Enzimaimunoensaio - ELISA : [',(probabilidade_falsoneg[3]  - erros_conservativos_falsoneg[3])*100,";",(probabilidade_falsoneg[3]  + erros_conservativos_falsoneg[3])*100,']')

print('Teste Quimioluminescência - CLIA : [',(probabilidade_falsoneg[4]  - erros_conservativos_falsoneg[4])*100,";",(probabilidade_falsoneg[4]  + erros_conservativos_falsoneg[4])*100,']')

print('Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : [',(probabilidade_falsoneg[5]  - erros_conservativos_falsoneg[5])*100,";",(probabilidade_falsoneg[5] + erros_conservativos_falsoneg[5])*100,']')

#Fazendo a análise para os casos falsos positivos

erros_otimistas_falsoposi = [erro_otimista(alpha, probabilidade_falsoposi[0], total_RT),erro_otimista(alpha, probabilidade_falsoposi[1], total_anticorpo),
                   erro_otimista(alpha, probabilidade_falsoposi[2], total_antigeno), erro_otimista(alpha, probabilidade_falsoposi[3], total_ELISA),
                   erro_otimista(alpha, probabilidade_falsoposi[4], total_CLIA), erro_otimista(alpha, probabilidade_falsoposi[5], total_ECLIA)]

#Estimativa intervalar para a abordagem otimista
print('\nConsiderando a abordagem otimista \n')

print('Teste RT-PCR : [',(probabilidade_falsoposi[0] - erros_otimistas_falsoposi[0])*100,";",(probabilidade_falsoposi[0] + erros_otimistas_falsoposi[0])*100,']')

print('Teste Rápido - Anticorpo : [',(probabilidade_falsoposi[1] - erros_otimistas_falsoposi[1])*100,";",(probabilidade_falsoposi[1] + erros_otimistas_falsoposi[1])*100,']')

print('Teste Rápido - Antígeno : [',(probabilidade_falsoposi[2] - erros_otimistas_falsoposi[2])*100,";",(probabilidade_falsoposi[2] + erros_otimistas_falsoposi[2])*100,']')

print('Teste Enzimaimunoensaio - ELISA : [',(probabilidade_falsoposi[3] - erros_otimistas_falsoposi[3])*100,";",(probabilidade_falsoposi[3] + erros_otimistas_falsoposi[3])*100,']')

print('Teste Quimioluminescência - CLIA : [',(probabilidade_falsoposi[4] - erros_otimistas_falsoposi[4])*100,";",(probabilidade_falsoposi[4] + erros_otimistas_falsoposi[4])*100,']')

print('Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : [',(probabilidade_falsoposi[5] - erros_otimistas_falsoposi[5])*100,";",(probabilidade_falsoposi[5] + erros_otimistas_falsoposi[5])*100,']')

erros_conservativos_falsoposi = [erro_conservativo(alpha, total_RT), erro_conservativo(alpha, total_anticorpo), 
                                erro_conservativo(alpha, total_antigeno), erro_conservativo(alpha, total_ELISA),
                                erro_conservativo(alpha, total_CLIA), erro_conservativo(alpha, total_ECLIA)]

#Estimativa intervalar para a abordagem conservativa
print('\nConsiderando a abordagem conservativa\n')

print('Teste RT-PCR : [',(probabilidade_falsoposi[0] - erros_conservativos_falsoposi[0])*100,";",(probabilidade_falsoposi[0] + erros_conservativos_falsoposi[0])*100,']')

print('Teste Rápido - Anticorpo : [',(probabilidade_falsoposi[1]  - erros_conservativos_falsoposi[1])*100,";",(probabilidade_falsoposi[1]  + erros_conservativos_falsoposi[1])*100,']')

print('Teste Rápido - Antígeno : [',(probabilidade_falsoposi[2]  - erros_conservativos_falsoposi[2])*100,";",(probabilidade_falsoposi[2] + erros_conservativos_falsoposi[2])*100,']')

print('Teste Enzimaimunoensaio - ELISA : [',(probabilidade_falsoposi[3]  - erros_conservativos_falsoposi[3])*100,";",(probabilidade_falsoposi[3]  + erros_conservativos_falsoposi[3])*100,']')

print('Teste Quimioluminescência - CLIA : [',(probabilidade_falsoposi[4]  - erros_conservativos_falsoposi[4])*100,";",(probabilidade_falsoposi[4]  + erros_conservativos_falsoposi[4])*100,']')

print('Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : [',(probabilidade_falsoposi[5]  - erros_conservativos_falsoposi[5])*100,";",(probabilidade_falsoposi[5] + erros_conservativos_falsoposi[5])*100,']')


Para os casos de falsos negativos

Considerando a abordagem otimista

Teste RT-PCR : [ 5.524870470080265 ; 5.604967054770309 ]
Teste Rápido - Anticorpo : [ 5.62759130661652 ; 5.756042287889019 ]
Teste Rápido - Antígeno : [ 4.5984850641572805 ; 4.832581112313307 ]
Teste Enzimaimunoensaio - ELISA : [ 7.999310001663637 ; 10.350689998336362 ]
Teste Quimioluminescência - CLIA : [ 3.458589589241121 ; 5.864275965224041 ]
Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : [ 1.1409540551883008 ; 1.9519972715250313 ]

Considerando a abordagem conservativa

Teste RT-PCR : [ 5.4775697672640735 ; 5.652267757586501 ]
Teste Rápido - Anticorpo : [ 5.553212310422811 ; 5.830421284082727 ]
Teste Rápido - Antígeno : [ 4.439438662530763 ; 4.991627513939826 ]
Teste Enzimaimunoensaio - ELISA : [ 7.138628134245059 ; 11.211371865754941 ]
Teste Quimioluminescência - CLIA : [ 1.808542991624176 ; 7.514322562840986 ]
Teste Imunoensaio por Eletroquimioluminescência - ECLIA IgG : [ -0.09674855192120385 ; 